In [1]:
pip install --user -U transformers accelerate sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import glob
import time
import pandas as pd
from docx import Document
from transformers import pipeline
from huggingface_hub import login

# ============================================
#  設定（ここを書き換えて使用）
# ============================================
HF_TOKEN = ""
MODEL_ID = "google/gemma-2-2b-it"
TARGET_CHARS = 120

# 各自変更
FOLDER = "/home/k101okab/k_data/jiiken5/5-3/PBL/tv"

# ============================================
#  HuggingFace ログイン
# ============================================
login(token=HF_TOKEN)

# ============================================
# モデル読み込み（GPU 0 を使用）
# ============================================
generator = pipeline(
    "text-generation",
    model=MODEL_ID,
    device=0,
    torch_dtype="auto",
    model_kwargs={"torch_dtype": "float16"}  # VRAM 使用量を半分に
)


print("✅ モデルロード完了\n")

# ============================================
# 📖 .docx 読み込み
# ============================================
def read_docx(path):
    try:
        doc = Document(path)
        return "\n".join([p.text.strip() for p in doc.paragraphs if p.text.strip()])
    except Exception as e:
        print(f"❌ 読み込みエラー: {path} → {e}")
        return ""

# ============================================
#  要約生成
# ============================================
def summarize(text: str, char_limit: int = TARGET_CHARS):

    prompt = (
        f"以下の文章を{char_limit}字前後（80〜140字）で自然に要約してください。\n"
        "文を途中で切らず、読みやすくまとめてください。\n\n"
        f"【入力】\n{text}\n\n【要約】"
    )

    result = generator(
        prompt,
        max_new_tokens=200,
        temperature=0.3,
        top_p=0.9,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=generator.tokenizer.eos_token_id
    )[0]["generated_text"]

    summary = result.replace(prompt, "").strip()

    # 改行・重複行削除
    lines = summary.split("\n")
    uniq = []
    for ln in lines:
        if ln.strip() and ln not in uniq:
            uniq.append(ln.strip())
    summary = " ".join(uniq)

    # 🔧 文単位で100字前後に調整（途中で切らない）
    import re
    sentences = re.split(r'(?<=[。！？])', summary)

    result_text = ""
    for s in sentences:
        # 入れたら140字を超えるなら終了
        if len(result_text) + len(s) > 140:
            break
        result_text += s

    # まだ短い（80字未満）なら元の summary をそのまま返す
    if len(result_text) < 80:
        result_text = summary

    return result_text.strip(), len(result_text.strip())


# ============================================
#  単一フォルダ内の docx を処理
# ============================================
def main():
    paths = glob.glob(os.path.join(FOLDER, "*.docx"))

    if not paths:
        print(f" .docx ファイルが見つかりません（フォルダ: {FOLDER}）")
        return

    results = []
    print(f" {len(paths)} 個の .docx ファイルを処理します\n")

    for i, path in enumerate(paths, 1):
        fname = os.path.basename(path)
        print(f"[{i}/{len(paths)}] {fname} 処理中…")

        text = read_docx(path)
        if not text:
            continue

        summary, slen = summarize(text)

        results.append({
            "ファイル名": fname,
            "本文冒頭100文字": text[:100],
            "要約": summary,
            "要約文字数": slen
        })

    timestamp = time.strftime("%Y%m%d_%H%M%S")
    out_csv = os.path.join(FOLDER, f"summary_{timestamp}.csv")

    pd.DataFrame(results).to_csv(out_csv, encoding="utf-8-sig", index=False)

    print(f"\n🎉 完了！CSV 出力 → {out_csv}")

if __name__ == "__main__":
    main()

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


✅ モデルロード完了

📁 74 個の .docx ファイルを処理します

[1/74] 210829.docx 処理中…
[2/74] 220501.docx 処理中…
[3/74] 230528-原稿２.docx 処理中…
[4/74] 200906.docx 処理中…
[5/74] 230917-原稿３.docx 処理中…
[6/74] 220410-もぎナレーション２.docx 処理中…
[7/74] 220109.docx 処理中…
[8/74] 221002-もぎナレ.docx 処理中…
[9/74] 230827-もぎナレ.docx 処理中…
[10/74] 230416-大洲NA案_2稿.docx 処理中…


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[11/74] 230402-原稿２.docx 処理中…
[12/74] 230723-原稿０２.docx 処理中…
[13/74] 220710-もぎナレ.docx 処理中…
[14/74] 221009-ナレ原稿２.docx 処理中…
[15/74] 220508-もぎナレ改.docx 処理中…
[16/74] 220605.docx 処理中…
[17/74] 220306.docx 処理中…
[18/74] 210704-もぎナレーション.docx 処理中…
[19/74] 210912-もぎナレーション.docx 処理中…
[20/74] 230709-ひんやりNA兵動-宮岡さん.docx 処理中…
[21/74] 230611-スーパーNA.docx 処理中…
[22/74] 200705-もぎナレーション.docx 処理中…
[23/74] 230730-もぎナレ.docx 処理中…
[24/74] 210110.docx 処理中…
[25/74] 211003.docx 処理中…
[26/74] 220417-もぎ大街道原稿３【確定】.docx 処理中…
[27/74] 200712.docx 処理中…
[28/74] 210509.docx 処理中…
[29/74] 210523-もぎナレーション.docx 処理中…
[30/74] 201206-もぎナレーション.docx 処理中…
[31/74] 200405-久万花桃の里ナレ.docx 処理中…
[32/74] 230226-原稿０３.docx 処理中…
[33/74] 210131-もぎ新店仮ナレ.docx 処理中…
[34/74] 210411.docx 処理中…
[35/74] 201101.docx 処理中…
[36/74] 220220-ナレ原稿新挑戦.docx 処理中…
[37/74] 230423-もぎ花桃の里ナレ台本0419_確.docx 処理中…
[38/74] 230903-西予市NA兵動荻山さん.docx 処理中…
[39/74] 221106-ナレ原稿３.docx 処理中…
[40/74] 230514-道後NA案(宮岡さんチェック).docx 処理中…
[41/74] 220814-もぎたてテレビ原稿【確定】.docx 処理中…
[42/74] 210307.docx 